## Import Libraries

In [1]:
import numpy as np
from numpy import where
import pandas as pd
import time

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from xgboost.sklearn import XGBRegressor

pd.options.display.max_columns = None
pd.options.display.max_rows = None
np.set_printoptions(threshold=np.inf)

import warnings
warnings.simplefilter(action='ignore')

## Predict TV

In [2]:
df = pd.read_csv('/Users/user/Documents/project/trade-price-ir-vegas.csv')

In [3]:
df.head()

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega
0,2022-09-02,dummyTrade1,USD,-100,-227907.098775,1y,2023-09-04,10y,1.962246
1,2022-09-02,dummyTrade1,USD,-50,-222208.400967,1y,2023-09-04,10y,-3.812341
2,2022-09-02,dummyTrade1,USD,-25,-218960.927995,1y,2023-09-04,10y,4.471006
3,2022-09-02,dummyTrade1,USD,-10,-216872.430106,1y,2023-09-04,10y,4.333398
4,2022-09-02,dummyTrade1,USD,-5,-216146.310328,1y,2023-09-04,10y,5.679687


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3919104 entries, 0 to 3919103
Data columns (total 9 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Value Date       object 
 1   Trade Name       object 
 2   Trade Currency   object 
 3   Zero Rate Shock  int64  
 4   TV               float64
 5   Expiry Bucket    object 
 6   Expiry Date      object 
 7   Tenor Bucket     object 
 8   Vega             float64
dtypes: float64(2), int64(1), object(6)
memory usage: 269.1+ MB


In [5]:
df['Value Date'] = pd.to_datetime(df['Value Date'])
df['Expiry Date'] = pd.to_datetime(df['Expiry Date'])
df= df.drop(['Trade Name', 'Trade Currency'],axis = 1)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3919104 entries, 0 to 3919103
Data columns (total 7 columns):
 #   Column           Dtype         
---  ------           -----         
 0   Value Date       datetime64[ns]
 1   Zero Rate Shock  int64         
 2   TV               float64       
 3   Expiry Bucket    object        
 4   Expiry Date      datetime64[ns]
 5   Tenor Bucket     object        
 6   Vega             float64       
dtypes: datetime64[ns](2), float64(2), int64(1), object(2)
memory usage: 209.3+ MB


,Value Date,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega
0,2022-09-02,-100,-227907.098775,1y,2023-09-04,10y,1.962246
1,2022-09-02,-50,-222208.400967,1y,2023-09-04,10y,-3.812341
2,2022-09-02,-25,-218960.927995,1y,2023-09-04,10y,4.471006
3,2022-09-02,-10,-216872.430106,1y,2023-09-04,10y,4.333398
4,2022-09-02,-5,-216146.310328,1y,2023-09-04,10y,5.679687


In [6]:
df['Value Year'] = df['Value Date'].dt.year
df['Value Month'] = df['Value Date'].dt.month
df['Value Day'] = df['Value Date'].dt.day
df['Expiry Year'] = df['Expiry Date'].dt.year
df['Expiry Month'] = df['Expiry Date'].dt.month
df['Expiry Day'] = df['Expiry Date'].dt.day
df = df.drop(['Value Date', 'Expiry Date'],axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3919104 entries, 0 to 3919103
Data columns (total 11 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Zero Rate Shock  int64  
 1   TV               float64
 2   Expiry Bucket    object 
 3   Tenor Bucket     object 
 4   Vega             float64
 5   Value Year       int32  
 6   Value Month      int32  
 7   Value Day        int32  
 8   Expiry Year      int32  
 9   Expiry Month     int32  
 10  Expiry Day       int32  
dtypes: float64(2), int32(6), int64(1), object(2)
memory usage: 239.2+ MB


In [7]:
df.describe()

,Zero Rate Shock,TV,Vega,Value Year,Value Month,Value Day,Expiry Year,Expiry Month,Expiry Day
count,3.919104e+06,3.919104e+06,3.919104e+06,3.919104e+06,3.919104e+06,3.919104e+06,3.919104e+06,3.919104e+06,3.919104e+06
mean,6.250000e+00,-3.471395e+05,1.714511e+01,2.022798e+03,7.473214e+00,1.561310e+01,2.028311e+03,7.314484e+00,1.609689e+01
std,5.136167e+01,1.856180e+05,2.850144e+03,4.700498e-01,3.474293e+00,8.903535e+00,4.410479e+00,3.474583e+00,8.879236e+00
min,-1.000000e+02,-7.174312e+05,-1.385180e+06,2.022000e+03,1.000000e+00,1.000000e+00,2.023000e+03,1.000000e+00,1.000000e+00
25%,-1.375000e+01,-5.291890e+05,-1.342167e+00,2.023000e+03,5.000000e+00,8.000000e+00,2.025000e+03,4.000000e+00,8.000000e+00
50%,2.500000e+00,-2.242142e+05,1.336000e-04,2.023000e+03,8.000000e+00,1.500000e+01,2.027000e+03,8.000000e+00,1.600000e+01
75%,3.125000e+01,-1.857137e+05,7.386112e+00,2.023000e+03,1.000000e+01,2.300000e+01,2.031000e+03,1.000000e+01,2.400000e+01
max,1.000000e+02,-2.911344e+04,1.110263e+06,2.024000e+03,1.200000e+01,3.100000e+01,2.039000e+03,1.200000e+01,3.100000e+01


In [8]:
le = LabelEncoder()
df['Expiry Bucket'] = le.fit_transform(df['Expiry Bucket'])
df['Tenor Bucket'] = le.fit_transform(df['Tenor Bucket'])
X = df.drop('TV',axis=1)
y = df['TV']
df.head()

,Zero Rate Shock,TV,Expiry Bucket,Tenor Bucket,Vega,Value Year,Value Month,Value Day,Expiry Year,Expiry Month,Expiry Day
0,-100,-227907.098775,3,0,1.962246,2022,9,2,2023,9,4
1,-50,-222208.400967,3,0,-3.812341,2022,9,2,2023,9,4
2,-25,-218960.927995,3,0,4.471006,2022,9,2,2023,9,4
3,-10,-216872.430106,3,0,4.333398,2022,9,2,2023,9,4
4,-5,-216146.310328,3,0,5.679687,2022,9,2,2023,9,4


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2024)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2743372, 10) (1175732, 10) (2743372,) (1175732,)


## Linear Regression模型及调参

调参参考链接 
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [10]:
lr = LinearRegression()
lr.fit(X_train,y_train)

param_grid = {'n_jobs':[2,10]}    
grid_search = GridSearchCV(lr, param_grid)    
grid_search.fit(X_train,y_train)
best_parameters = grid_search.best_estimator_.get_params()    
for para, val in list(best_parameters.items()):    
    print(para, val)
best_lr_model = LinearRegression(n_jobs=best_parameters['n_jobs']).fit(X_train,y_train)
preds1 = best_lr_model.predict(X_test)

print('LinearRegression Test r2:',best_lr_model.score(X_test,y_test))

copy_X True
fit_intercept True
n_jobs 2
positive False
LinearRegression Test r2: 0.09788330025932113


## Random Forest模型及调参

调参参考链接：
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

In [11]:
rf = RandomForestRegressor(min_samples_leaf = 0.5, random_state=0)
rf.fit(X_train,y_train)

param_grid = {'n_estimators':[2,5]}    
grid_search = GridSearchCV(rf, param_grid, n_jobs = 1, verbose=10)    
grid_search.fit(X_train,y_train)
best_parameters = grid_search.best_estimator_.get_params()    
for para, val in list(best_parameters.items()):    
    print(para, val)
best_rf_model = RandomForestRegressor( n_estimators=best_parameters['n_estimators'],min_samples_leaf = 0.5, random_state=0).fit(X_train,y_train)
preds2 = best_rf_model.predict(X_test)

print('RandomForestRegressor Test r2:',best_rf_model.score(X_test,y_test))

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5; 1/2] START n_estimators=2..............................................
[CV 1/5; 1/2] END ..............n_estimators=2;, score=-0.000 total time=   0.8s
[CV 2/5; 1/2] START n_estimators=2..............................................
[CV 2/5; 1/2] END ..............n_estimators=2;, score=-0.000 total time=   0.8s
[CV 3/5; 1/2] START n_estimators=2..............................................
[CV 3/5; 1/2] END ..............n_estimators=2;, score=-0.000 total time=   0.8s
[CV 4/5; 1/2] START n_estimators=2..............................................
[CV 4/5; 1/2] END ..............n_estimators=2;, score=-0.000 total time=   0.8s
[CV 5/5; 1/2] START n_estimators=2..............................................
[CV 5/5; 1/2] END ..............n_estimators=2;, score=-0.000 total time=   0.7s
[CV 1/5; 2/2] START n_estimators=5..............................................
[CV 1/5; 2/2] END ..............n_estimators=5;, 

## XGBoost模型及调参

调参参考链接：
https://xgboost.readthedocs.io/en/latest/parameter.html

In [12]:
xgb = XGBRegressor(random_state=0)
xgb.fit(X_train,y_train)

param_grid = {'max_depth':[4,6],'min_child_weight':[1,2]}    
grid_search = GridSearchCV(xgb, param_grid, n_jobs = 1, verbose=10)    
grid_search.fit(X_train,y_train)
best_parameters = grid_search.best_estimator_.get_params()    
for para, val in list(best_parameters.items()):    
    print(para, val)
best_xgb_model = XGBRegressor(max_depth=best_parameters['max_depth'], 
                      min_child_weight=best_parameters['min_child_weight'],random_state=0).fit(X_train,y_train)
preds3 = best_xgb_model.predict(X_test)
print('XGB Test r2:',best_xgb_model.score(X_test,y_test))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5; 1/4] START max_depth=4, min_child_weight=1.............................
[CV 1/5; 1/4] END max_depth=4, min_child_weight=1;, score=0.431 total time=  10.2s
[CV 2/5; 1/4] START max_depth=4, min_child_weight=1.............................
[CV 2/5; 1/4] END max_depth=4, min_child_weight=1;, score=0.431 total time=  10.6s
[CV 3/5; 1/4] START max_depth=4, min_child_weight=1.............................
[CV 3/5; 1/4] END max_depth=4, min_child_weight=1;, score=0.433 total time=  10.9s
[CV 4/5; 1/4] START max_depth=4, min_child_weight=1.............................
[CV 4/5; 1/4] END max_depth=4, min_child_weight=1;, score=0.431 total time=  11.3s
[CV 5/5; 1/4] START max_depth=4, min_child_weight=1.............................
[CV 5/5; 1/4] END max_depth=4, min_child_weight=1;, score=0.434 total time=  11.4s
[CV 1/5; 2/4] START max_depth=4, min_child_weight=2.............................
[CV 1/5; 2/4] END max_depth=4, min_chil

## Predict Vega

In [13]:
df = pd.read_csv('/Users/user/Documents/project/trade-price-ir-vegas.csv')

In [14]:
df.head()

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega
0,2022-09-02,dummyTrade1,USD,-100,-227907.098775,1y,2023-09-04,10y,1.962246
1,2022-09-02,dummyTrade1,USD,-50,-222208.400967,1y,2023-09-04,10y,-3.812341
2,2022-09-02,dummyTrade1,USD,-25,-218960.927995,1y,2023-09-04,10y,4.471006
3,2022-09-02,dummyTrade1,USD,-10,-216872.430106,1y,2023-09-04,10y,4.333398
4,2022-09-02,dummyTrade1,USD,-5,-216146.310328,1y,2023-09-04,10y,5.679687


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3919104 entries, 0 to 3919103
Data columns (total 9 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Value Date       object 
 1   Trade Name       object 
 2   Trade Currency   object 
 3   Zero Rate Shock  int64  
 4   TV               float64
 5   Expiry Bucket    object 
 6   Expiry Date      object 
 7   Tenor Bucket     object 
 8   Vega             float64
dtypes: float64(2), int64(1), object(6)
memory usage: 269.1+ MB


In [16]:
df['Value Date'] = pd.to_datetime(df['Value Date'])
df['Expiry Date'] = pd.to_datetime(df['Expiry Date'])
df= df.drop(['Trade Name', 'Trade Currency'],axis = 1)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3919104 entries, 0 to 3919103
Data columns (total 7 columns):
 #   Column           Dtype         
---  ------           -----         
 0   Value Date       datetime64[ns]
 1   Zero Rate Shock  int64         
 2   TV               float64       
 3   Expiry Bucket    object        
 4   Expiry Date      datetime64[ns]
 5   Tenor Bucket     object        
 6   Vega             float64       
dtypes: datetime64[ns](2), float64(2), int64(1), object(2)
memory usage: 209.3+ MB


,Value Date,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega
0,2022-09-02,-100,-227907.098775,1y,2023-09-04,10y,1.962246
1,2022-09-02,-50,-222208.400967,1y,2023-09-04,10y,-3.812341
2,2022-09-02,-25,-218960.927995,1y,2023-09-04,10y,4.471006
3,2022-09-02,-10,-216872.430106,1y,2023-09-04,10y,4.333398
4,2022-09-02,-5,-216146.310328,1y,2023-09-04,10y,5.679687


In [17]:
df['Value Year'] = df['Value Date'].dt.year
df['Value Month'] = df['Value Date'].dt.month
df['Value Day'] = df['Value Date'].dt.day
df['Expiry Year'] = df['Expiry Date'].dt.year
df['Expiry Month'] = df['Expiry Date'].dt.month
df['Expiry Day'] = df['Expiry Date'].dt.day
df = df.drop(['Value Date', 'Expiry Date'],axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3919104 entries, 0 to 3919103
Data columns (total 11 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Zero Rate Shock  int64  
 1   TV               float64
 2   Expiry Bucket    object 
 3   Tenor Bucket     object 
 4   Vega             float64
 5   Value Year       int32  
 6   Value Month      int32  
 7   Value Day        int32  
 8   Expiry Year      int32  
 9   Expiry Month     int32  
 10  Expiry Day       int32  
dtypes: float64(2), int32(6), int64(1), object(2)
memory usage: 239.2+ MB


In [18]:
df.describe()

,Zero Rate Shock,TV,Vega,Value Year,Value Month,Value Day,Expiry Year,Expiry Month,Expiry Day
count,3.919104e+06,3.919104e+06,3.919104e+06,3.919104e+06,3.919104e+06,3.919104e+06,3.919104e+06,3.919104e+06,3.919104e+06
mean,6.250000e+00,-3.471395e+05,1.714511e+01,2.022798e+03,7.473214e+00,1.561310e+01,2.028311e+03,7.314484e+00,1.609689e+01
std,5.136167e+01,1.856180e+05,2.850144e+03,4.700498e-01,3.474293e+00,8.903535e+00,4.410479e+00,3.474583e+00,8.879236e+00
min,-1.000000e+02,-7.174312e+05,-1.385180e+06,2.022000e+03,1.000000e+00,1.000000e+00,2.023000e+03,1.000000e+00,1.000000e+00
25%,-1.375000e+01,-5.291890e+05,-1.342167e+00,2.023000e+03,5.000000e+00,8.000000e+00,2.025000e+03,4.000000e+00,8.000000e+00
50%,2.500000e+00,-2.242142e+05,1.336000e-04,2.023000e+03,8.000000e+00,1.500000e+01,2.027000e+03,8.000000e+00,1.600000e+01
75%,3.125000e+01,-1.857137e+05,7.386112e+00,2.023000e+03,1.000000e+01,2.300000e+01,2.031000e+03,1.000000e+01,2.400000e+01
max,1.000000e+02,-2.911344e+04,1.110263e+06,2.024000e+03,1.200000e+01,3.100000e+01,2.039000e+03,1.200000e+01,3.100000e+01


In [19]:
le = LabelEncoder()
df['Expiry Bucket'] = le.fit_transform(df['Expiry Bucket'])
df['Tenor Bucket'] = le.fit_transform(df['Tenor Bucket'])
X = df.drop('Vega',axis=1)
y = df['Vega']
df.head()

,Zero Rate Shock,TV,Expiry Bucket,Tenor Bucket,Vega,Value Year,Value Month,Value Day,Expiry Year,Expiry Month,Expiry Day
0,-100,-227907.098775,3,0,1.962246,2022,9,2,2023,9,4
1,-50,-222208.400967,3,0,-3.812341,2022,9,2,2023,9,4
2,-25,-218960.927995,3,0,4.471006,2022,9,2,2023,9,4
3,-10,-216872.430106,3,0,4.333398,2022,9,2,2023,9,4
4,-5,-216146.310328,3,0,5.679687,2022,9,2,2023,9,4


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2024)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2743372, 10) (1175732, 10) (2743372,) (1175732,)


## Linear Regression模型及调参

调参参考链接 
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [21]:
lr = LinearRegression()
lr.fit(X_train,y_train)

param_grid = {'n_jobs':[2,10]}    
grid_search = GridSearchCV(lr, param_grid)    
grid_search.fit(X_train,y_train)
best_parameters = grid_search.best_estimator_.get_params()    
for para, val in list(best_parameters.items()):    
    print(para, val)
best_lr_model = LinearRegression(n_jobs=best_parameters['n_jobs']).fit(X_train,y_train)
preds1 = best_lr_model.predict(X_test)

print('LinearRegression Test r2:',best_lr_model.score(X_test,y_test))

copy_X True
fit_intercept True
n_jobs 2
positive False
LinearRegression Test r2: 2.8990077477586418e-05


## Random Forest模型及调参

调参参考链接：
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

In [22]:
rf = RandomForestRegressor(min_samples_leaf = 0.5,random_state=0)
rf.fit(X_train,y_train)

param_grid = {'n_estimators':[2,5]}    
grid_search = GridSearchCV(rf, param_grid, n_jobs = 1, verbose=10)    
grid_search.fit(X_train,y_train)
best_parameters = grid_search.best_estimator_.get_params()    
for para, val in list(best_parameters.items()):    
    print(para, val)
best_rf_model = RandomForestRegressor( n_estimators=best_parameters['n_estimators'],min_samples_leaf = 0.5,random_state=0).fit(X_train,y_train)
preds2 = best_rf_model.predict(X_test)

print('RandomForestRegressor Test r2:',best_rf_model.score(X_test,y_test))

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5; 1/2] START n_estimators=2..............................................
[CV 1/5; 1/2] END ..............n_estimators=2;, score=-0.000 total time=   0.8s
[CV 2/5; 1/2] START n_estimators=2..............................................
[CV 2/5; 1/2] END ..............n_estimators=2;, score=-0.000 total time=   0.7s
[CV 3/5; 1/2] START n_estimators=2..............................................
[CV 3/5; 1/2] END ..............n_estimators=2;, score=-0.000 total time=   0.8s
[CV 4/5; 1/2] START n_estimators=2..............................................
[CV 4/5; 1/2] END ..............n_estimators=2;, score=-0.000 total time=   0.8s
[CV 5/5; 1/2] START n_estimators=2..............................................
[CV 5/5; 1/2] END ..............n_estimators=2;, score=-0.000 total time=   0.8s
[CV 1/5; 2/2] START n_estimators=5..............................................
[CV 1/5; 2/2] END ..............n_estimators=5;, 

## XGBoost模型及调参

调参参考链接：
https://xgboost.readthedocs.io/en/latest/parameter.html

In [23]:
xgb = XGBRegressor(random_state=0)
xgb.fit(X_train,y_train)

param_grid = {'max_depth':[4,6],'min_child_weight':[1,2]}    
grid_search = GridSearchCV(xgb, param_grid, n_jobs = 1, verbose=10)    
grid_search.fit(X_train,y_train)
best_parameters = grid_search.best_estimator_.get_params()    
for para, val in list(best_parameters.items()):    
    print(para, val)
best_xgb_model = XGBRegressor(max_depth=best_parameters['max_depth'], 
                      min_child_weight=best_parameters['min_child_weight'],random_state=0).fit(X_train,y_train)
preds3 = best_xgb_model.predict(X_test)
print('XGB Test r2:',best_xgb_model.score(X_test,y_test))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5; 1/4] START max_depth=4, min_child_weight=1.............................
[CV 1/5; 1/4] END max_depth=4, min_child_weight=1;, score=0.011 total time=   9.9s
[CV 2/5; 1/4] START max_depth=4, min_child_weight=1.............................
[CV 2/5; 1/4] END max_depth=4, min_child_weight=1;, score=0.013 total time=  10.4s
[CV 3/5; 1/4] START max_depth=4, min_child_weight=1.............................
[CV 3/5; 1/4] END max_depth=4, min_child_weight=1;, score=0.014 total time=  10.9s
[CV 4/5; 1/4] START max_depth=4, min_child_weight=1.............................
[CV 4/5; 1/4] END max_depth=4, min_child_weight=1;, score=0.015 total time=  11.0s
[CV 5/5; 1/4] START max_depth=4, min_child_weight=1.............................
[CV 5/5; 1/4] END max_depth=4, min_child_weight=1;, score=0.012 total time=  11.2s
[CV 1/5; 2/4] START max_depth=4, min_child_weight=2.............................
[CV 1/5; 2/4] END max_depth=4, min_chil

__________________________________